In [1]:
import os, sys
from glob import glob
import numpy as np
import dask
import xarray as xr
import xgcm
import cartopy.crs as ccrs
from cmocean import cm

from matplotlib import pyplot as plt
%matplotlib inline

from mitequinox.utils import *
from mitequinox.sigp import *
from mitequinox.plot import *
from dask import compute, delayed

In [2]:
from dask_jobqueue import PBSCluster
cluster = PBSCluster(cores=12, walltime='03:00:00')
w = cluster.scale(10*10)

In [3]:
# get dask handles and check dask server status
from dask.distributed import Client
client = Client(cluster)

In [4]:
client

Client Scheduler: tcp://10.135.39.22:42088 Dashboard: http://10.135.39.22:8787/status,Cluster Workers: 36 Cores: 36 Memory: 299.88 GB


__________

# Combine Eta and V together, and then store

In [5]:
time_length = 1440
face = 1
dij=4

In [16]:
# 3.5 mins for 2 months (1440)
overwrite=True
    
#for face in range(13):
for face in [1]:

    Efile = work_data_dir+'xy/comparison/Movies/Eta_SSV_f%02d.zarr'%(face)

    if not os.path.isdir(Efile) or overwrite:

        # load data
        dsE = xr.open_zarr(root_data_dir+'zarr/%s.zarr'%('Eta')).isel(time=slice(1512+200,1512+200+time_length))
        dsE = dsE.isel(face = face, i=slice(0,None,dij), j=slice(0,None,dij))
        
        ds_v = xr.open_zarr(work_data_dir+'xy/comparison/v_f%02d.zarr'%(face))
        dsv = ds_v.isel(time=slice(200,200+time_length))
        del dsv['v'].encoding['chunks']
        del dsv['v_geo'].encoding['chunks']
        del dsv['v_ageo'].encoding['chunks']
        dsv = dsv.chunk({'time':1, 'i': 24*8, 'j':47*4})    
        
        E = xr.merge([dsE,dsv])

        # store
        for c in E.coords:
            try:
                del E[c].encoding['chunks']
            except:
                print(c)

        E = E.chunk({'time':1, 'i': 1080, 'j':1080})
        print(E)
        %time E.to_zarr(Efile, mode='w')
        print('--- face %d done'%face)

    else:
        print('--- face %d allready computed'%face)

distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)


face


distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)


<xarray.Dataset>
Dimensions:  (i: 1080, j: 1080, time: 1440)
Coordinates:
    dtime    (time) datetime64[ns] dask.array<chunksize=(1,), meta=np.ndarray>
    face     int64 1
  * i        (i) int64 0 4 8 12 16 20 24 ... 4292 4296 4300 4304 4308 4312 4316
    iters    (time) int64 dask.array<chunksize=(1,), meta=np.ndarray>
  * j        (j) int64 0 4 8 12 16 20 24 ... 4292 4296 4300 4304 4308 4312 4316
  * time     (time) float64 6.422e+06 6.426e+06 6.43e+06 ... 1.16e+07 1.16e+07
    CS       (j, i) float32 dask.array<chunksize=(1080, 1080), meta=np.ndarray>
    Depth    (j, i) float32 dask.array<chunksize=(1080, 1080), meta=np.ndarray>
    SN       (j, i) float32 dask.array<chunksize=(1080, 1080), meta=np.ndarray>
    XC       (j, i) float32 dask.array<chunksize=(1080, 1080), meta=np.ndarray>
    YC       (j, i) float32 dask.array<chunksize=(1080, 1080), meta=np.ndarray>
    dxC      (j, i) float32 dask.array<chunksize=(1080, 1080), meta=np.ndarray>
    dxG      (j, i) float32 dask.arra

distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

CPU times: user 3min 1s, sys: 6.63 s, total: 3min 8s
Wall time: 3min 16s
--- face 1 done


In [6]:
# read data
ds_ice = xr.open_zarr(work_data_dir+'xy/sea_ice_mask.zarr')
ice = ds_ice.AREA.isel(i=slice(0,None,dij), j=slice(0,None,dij), face=face)

ds = xr.open_zarr(work_data_dir+'xy/comparison/Movies/Eta_SSV_f%02d.zarr'%(face))
# define (real) time
def iters_to_date(iters, delta_t=3600.):
    t0 = datetime.datetime(2011,11,23,8)    
    ltime = delta_t * (np.array(iters))
    dtime = [t0+dateutil.relativedelta.relativedelta(seconds=t) for t in ltime]    
    return dtime

time_day = iters_to_date(np.arange(time_length))
ds = ds.assign_coords(time=time_day) 
print(ds)
print('\n data size: %.1f GB' %(ds.nbytes / 1e9))

<xarray.Dataset>
Dimensions:  (i: 1080, j: 1080, time: 1440)
Coordinates:
    CS       (j, i) float32 dask.array<chunksize=(1080, 1080), meta=np.ndarray>
    Depth    (j, i) float32 dask.array<chunksize=(1080, 1080), meta=np.ndarray>
    SN       (j, i) float32 dask.array<chunksize=(1080, 1080), meta=np.ndarray>
    XC       (j, i) float32 dask.array<chunksize=(1080, 1080), meta=np.ndarray>
    YC       (j, i) float32 dask.array<chunksize=(1080, 1080), meta=np.ndarray>
    dtime    (time) datetime64[ns] dask.array<chunksize=(1440,), meta=np.ndarray>
    dxC      (j, i) float32 dask.array<chunksize=(1080, 1080), meta=np.ndarray>
    dxG      (j, i) float32 dask.array<chunksize=(1080, 1080), meta=np.ndarray>
    dyC      (j, i) float32 dask.array<chunksize=(1080, 1080), meta=np.ndarray>
    dyG      (j, i) float32 dask.array<chunksize=(1080, 1080), meta=np.ndarray>
    f_i      (j, i) float32 dask.array<chunksize=(1080, 1080), meta=np.ndarray>
    f_j      (j, i) float32 dask.array<chunk

# Movies

In [7]:
font_size = 12
linewidith = 0.5

def plot_scalar(v1, v2, v3, v4, colorbar=False, title=None, label=None, vmin=None, vmax=None, savefig=None, 
                offline=False, figsize=(10,10), cmap=None):
    #
    if vmin is None:
        vmin = v.min()
    if vmax is None:
        vmax = v.max()
    #
    MPL_LOCK = threading.Lock()
    with MPL_LOCK:
        if offline:
            plt.switch_backend('agg')
        #
        fig = plt.figure(figsize=figsize)
        #ax = fig.add_subplot(111, projection=ccrs.PlateCarree())
        #colmap = _get_cmap(v1, cmap)
        try:
            
            ## subplot 221
            ax = fig.add_subplot(221, projection=ccrs.PlateCarree())
            im = v1.plot.pcolormesh(ax=ax, transform=ccrs.PlateCarree(), vmin=-4, vmax=4,
                                   x='XC', y='YC', add_colorbar=colorbar, cmap=cmap)
            cb=fig.colorbar(im)
            #gl=ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=linewidith , color='k', alpha=0.5, linestyle='--')
            #gl.xlabels_top = False
            #gl.ylabels_right = False
            ax.grid(linewidth=linewidith , color='k', alpha=0.5, linestyle='--')
            ax.set_title(title[0],fontsize=font_size) 
            cb.set_label(label=label[0], fontsize=font_size)
            ax.set_xlim(-35., 50.)
            ax.set_xticks([-30, -15, 0, 15, 30, 45])
            ax.set_xticklabels(['$30\degree$W','$15\degree$W','$0\degree$','$15\degree$E','$30\degree$E','$45\degree$E'])
            ax.set_ylim(-55., 15.)
            ax.set_yticks([-45, -30, -15, 0, 15])
            ax.set_yticklabels(['$45\degree$S','$30\degree$S','$15\degree$S','$0\degree$','$15\degree$N'])
            ax.tick_params(direction='out', length=4, width=1.5)
            ax.set_ylabel('',fontsize=font_size)
            ax.set_xlabel('',fontsize=font_size)
            ax.tick_params(labelsize=font_size)

            ## subplot 222
            ax = fig.add_subplot(222, projection=ccrs.PlateCarree())
            im = v2.plot.pcolormesh(ax=ax, transform=ccrs.PlateCarree(), vmin=vmin, vmax=vmax,
                                   x='XC', y='YC', add_colorbar=colorbar, cmap=cmap)
            cb=fig.colorbar(im)
            ax.grid(linewidth=linewidith , color='k', alpha=0.5, linestyle='--')
            ax.set_title(title[1],fontsize=font_size)                        
            cb.set_label(label=label[1], fontsize=font_size)
            ax.set_xlim(-35., 50.)
            ax.set_xticks([-30, -15, 0, 15, 30, 45])
            ax.set_xticklabels(['$30\degree$W','$15\degree$W','$0\degree$','$15\degree$E','$30\degree$E','$45\degree$E'])
            ax.set_ylim(-55., 15.)
            ax.set_yticks([-45, -30, -15, 0, 15])
            ax.set_yticklabels(['$45\degree$S','$30\degree$S','$15\degree$S','$0\degree$','$15\degree$N'])
            ax.tick_params(direction='out', length=6, width=2)
            ax.set_ylabel('',fontsize=font_size)
            ax.set_xlabel('',fontsize=font_size)
            ax.tick_params(labelsize=font_size)
       
            ## subplot 223
            ax = fig.add_subplot(223, projection=ccrs.PlateCarree())
            im = v3.plot.pcolormesh(ax=ax, transform=ccrs.PlateCarree(), vmin=vmin, vmax=vmax,
                                   x='XC', y='YC', add_colorbar=colorbar, cmap=cmap)
            cb=fig.colorbar(im)
            ax.grid(linewidth=linewidith , color='k', alpha=0.5, linestyle='--')
            ax.set_title(title[2],fontsize=font_size)                        
            cb.set_label(label=label[2], fontsize=font_size)
            ax.set_xlim(-35., 50.)
            ax.set_xticks([-30, -15, 0, 15, 30, 45])
            ax.set_xticklabels(['$30\degree$W','$15\degree$W','$0\degree$','$15\degree$E','$30\degree$E','$45\degree$E'])
            ax.set_ylim(-55., 15.)
            ax.set_yticks([-45, -30, -15, 0, 15])
            ax.set_yticklabels(['$45\degree$S','$30\degree$S','$15\degree$S','$0\degree$','$15\degree$N'])
            ax.tick_params(direction='out', length=6, width=2)
            ax.set_ylabel('',fontsize=font_size)
            ax.set_xlabel('',fontsize=font_size)
            ax.tick_params(labelsize=font_size)
            
            ## subplot 224
            ax = fig.add_subplot(224, projection=ccrs.PlateCarree())
            im = v4.plot.pcolormesh(ax=ax, transform=ccrs.PlateCarree(), vmin=vmin, vmax=vmax,
                                   x='XC', y='YC', add_colorbar=colorbar, cmap=cmap)
            cb=fig.colorbar(im)
            ax.grid(linewidth=linewidith , color='k', alpha=0.5, linestyle='--')
            ax.set_title(title[3],fontsize=font_size)    
            cb.set_label(label=label[3], fontsize=font_size)
            ax.set_xlim(-35., 50.)
            ax.set_xticks([-30, -15, 0, 15, 30, 45])
            ax.set_xticklabels(['$30\degree$W','$15\degree$W','$0\degree$','$15\degree$E','$30\degree$E','$45\degree$E'])
            ax.set_ylim(-55., 15.)
            ax.set_yticks([-45, -30, -15, 0, 15])
            ax.set_yticklabels(['$45\degree$S','$30\degree$S','$15\degree$S','$0\degree$','$15\degree$N'])
            ax.tick_params(direction='out', length=6, width=2)
            ax.set_ylabel('',fontsize=font_size)
            ax.set_xlabel('',fontsize=font_size)
            ax.tick_params(labelsize=font_size)
                        
        except:
            pass
        #
        #if label is not None:
        #    ax.set_label(label=label, size=font_size)     #
            
        if savefig is not None:
            fig.savefig(savefig, dpi=150)
            plt.close(fig)
        #
        if not offline:
            plt.show()

In [8]:
vmin, vmax = -2, 2
lds = ds

def process(ds, i, overwrite=True):    
    
    v = ds['v'].where(ice>0)
    vg = ds['v_geo'].where(ice>0)
    va = ds['v_ageo'].where(ice>0)
    eta = ds['Eta'].where(ice>0)
    
    #mtime = ds['time'].values/86400
    mtime = time_day[i]
    figname = '/home/uz/yux/mit_equinox/hal/Geostrophy_assessment/Figures/V_Movies/All_V_f%02d_t%05d' %(face, i)+'.png'
    #
    if not os.path.isfile(figname) or overwrite:
        #
        #title = ['$\eta$ (m) %s' %mtime, 'u (m/s) %s' %mtime, '$u_g$ (m/s) %s' %mtime, '$u_a$ (m/s) %s' %mtime]
        title = ['%s' %mtime, '%s' %mtime, '%s' %mtime, '%s' %mtime]
        label = ['$\eta$ (m)', 'v (m/s)', '$v_g$ (m/s)', '$v_a$ (m/s)']
            
        plot_scalar(eta, v, vg, va, colorbar=False, title=title, label=label, savefig=figname, vmin=vmin, vmax=vmax,
                    offline=False, figsize=(15,10),cmap=cm.balance)
        #
        m = float(v.mean().values)
    else:
        m = -1.
    return m

I = range(len(ds['time']))
#I = range(0,10)
print(I)
values = [delayed(process)(lds.isel(time=i), i) for i in I]

range(0, 1440)


In [9]:
futures = client.compute(values)
%time results = client.gather(futures)

CPU times: user 2min 7s, sys: 44.1 s, total: 2min 51s
Wall time: 3min 28s


In [11]:
cluster.close()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/home/uz/yux/.conda/envs/equinox/lib/python3.7/site-packages/distributed/utils.py", line 666, in log_errors
    yield
  File "/home/uz/yux/.conda/envs/equinox/lib/python3.7/site-packages/distributed/client.py", line 1276, in _close
    await gen.with_timeout(timedelta(seconds=2), list(coroutines))
concurrent.futures._base.CancelledError
distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/home/uz/yux/.conda/envs/equinox/lib/python3.7/site-packages/distributed/utils.py", line 666, in log_errors
    yield
  File "/home/uz/yux/.conda/envs/equinox/lib/python3.7/site-packages/distributed/client.py", line 1005, in _reconnect
    await self._close()
  File "/home/uz/yux/.conda/envs/equinox/lib/python3.7/site-packages/distributed/client.py", line 1276, in _close
    await gen.with_timeout(timedelta(seconds=